---
# Demo - Safety and Security (Video Surveillance)

![overview](safety_and_security_overview.png)

### Overview

- Live video feeds from 4 cameras are continuously ingested into Streaming Data Platform (Pravega).

- Pravega asynchronously writes all data to Long Term Storage (ECS, Isilon, or HDFS) for long-term retention.

- Historical and live video can be played back using a simple example application (Jupyter Notebook).

### Benefits of using Pravega vs. a NAS-only solution

-  Pravega writes data *asynchronously* to Long Term Storage (ECS, Isilon, or HDFS) in large blocks, 
   which means that cost-effective magnetic media can be used.
   Without Pravega, all writes must go to Long Term Storage immediately, requiring either expensive low-latency hardware
   or reducing durability guarantees.
   
-  Video frames are written to Pravega atomically. Pravega guarantees that the stream will not be corrupted with partial frames.
   Without Pravega, writes to SMB and NFS are generally not atomic.
   
-  Pravega does not make a video frame is available to readers until the entire video frame has been durably persisted.

-  Pravega provides low end-to-end latency (15-20 milliseconds) between a camera writing a frame and a video player displaying it.
   This is less than 1 frame at 60 frames per second.
   **This low latency is impossible to achieve with file-only workflows due to limitations of locking, polling, NFS, and SMB.**
   
-  Older frames of video streams can be automatically truncated based on time or size limits.

-  The same API can be used to read live video or historical data from years ago, regardless of Long Term Storage technology.
   Pravega will serve data cached in memory if available or it will read from Long Term Storage.

### About the video feeds

To avoid the need for a set of live cameras for this demo, we play back video from a set of JPEG files on disk
and write each video frame to Pravega.

- We simulate 4 cameras including drone and ground cameras.
- All video frames are 1280 x 720 pixels, RGB color, JPEG format.
- Video frames are played back at 15 frames per second.
- The average frame size is 115 KB.
- The total ingest rate is 9.1 MB/sec.

All videos are from [Pexel](https://www.pexels.com) and are free to use.

---

### How to use this notebook
1. [Install dependencies](install_dependencies.ipynb).
2. [safety_and_security_download.ipynb](safety_and_security_download.ipynb).
3. [safety_and_security_transcode_from_video.ipynb](safety_and_security_transcode_from_video.ipynb).
4. [Begin ingesting data](safety_and_security_ingest.ipynb).
5. [Run continuous index builder job](safety_and_security_index_builder.ipynb).
6. Run video player. Click *Kernel* -> *Restart Kernel and Run All Cells*.

### Initialize

In [ ]:
#### Import dependencies
%load_ext autoreload
%autoreload 2
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import VideoPlayer
import os
imp.reload(video);

### Define Pravega stream parameters

gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
scope = 'examples'
stream = 'safety-and-security-video'

### Initialize connection to Pravega GRPC Gateway

pravega_channel = grpc.insecure_channel(gateway, options=[
        ('grpc.max_receive_message_length', 9*1024*1024),
    ])
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

---
### Video Player

In [ ]:
VideoPlayer(pravega_client=pravega_client, scope=scope, stream=stream).interact()